In [25]:
import os 
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero

dir_ = '../data/'
group_dir_ = '../data/groups/low'

In [26]:
file_name = 'normalized_to_rating_filter_track_5_user_100.csv'
svd = pd.read_pickle(os.path.join(dir_, 'prediction_svd_top_N_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])

file_name = 'normalized_popularity_filter_track_5_user_100.csv'
pop_count = pd.read_pickle(os.path.join(dir_, 'prediction_popularity_count_top_N_' + file_name[:-3] + 'pkl'))

num_user = len(svd['uid'].unique())
num_user

120

In [27]:
groups_n =[]
group_sizes = ['2', '3', '4', '5']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, 'group_' + group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)

In [28]:
for i in groups_n:
    print(len(i[0]))

2
3
4
5


In [29]:
svd['count'] = 1
svd

,uid,tid,rating,count
0,0,0,1.715628,1
1,0,5,1.818083,1
2,0,7,2.199155,1
3,0,8,2.362023,1
4,0,9,2.480099,1
...,...,...,...,...
3032324,119,26432,2.205485,1
3032325,119,26433,1.915779,1
3032326,119,26434,1.595435,1
3032327,119,26436,2.142235,1


In [30]:
pop_count = pop_count.sort_values(by=['count'],  ascending=False)
pop_count

,tid,count,rating
821,823,1201,210.25
3143,3158,814,88.25
710,712,719,114.25
904,907,652,59.25
481,483,627,98.00
...,...,...,...
14940,15045,1,0.25
6991,7029,1,0.25
13789,13880,1,0.25
23337,23542,1,0.25


In [31]:
# min_max
max_rating = pop_count.iloc[0]['count']
pop_normalized = pop_count.copy()
pop_normalized = pop_normalized[['tid', 'count']]
pop_normalized['rating'] = pop_normalized['count']
pop_normalized['rating'] /= max_rating
pop_normalized['rating'] *= 5
pop_normalized

,tid,count,rating
821,823,1201,5.000000
3143,3158,814,3.388843
710,712,719,2.993339
904,907,652,2.714405
481,483,627,2.610325
...,...,...,...
14940,15045,1,0.004163
6991,7029,1,0.004163
13789,13880,1,0.004163
23337,23542,1,0.004163


In [32]:
lambdas = []
for i in range(11):
    lambdas.append(i/10)
lambdas

[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

In [33]:
for lambda_ in tqdm(lambdas):
    top_n_size = 4
    top_n_lists_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_lists = []

        for group in groups:

            predictions = []
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
                predictions.append(prediction)
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = pop_normalized.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
    #         rating_table = rating_table[rating_table['count'] == group_size]
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
        top_n_lists_g.append(top_n_lists)   
        
    print('lambda = ' + str(lambda_))
    threshold = 0.0
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        precisions = []
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j]

            high_rating = 0

            for k in range(top_n_size):
                for l in range(len(group)):
                    uid = group[l]
                    tid = top_n_list[k]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    if len(t) > 0 and t.iloc[0]['rating'] > threshold:
                        high_rating += 1
                        break

            precision = high_rating / top_n_size
            precisions.append(precision)

        avg_precision = 0
        for precision in precisions:
            avg_precision += precision
        avg_precision /= len(precisions)
        print(avg_precision)
    print('----------------------------------------') 

lambda = 0.0
0.38333333333333336
0.4722222222222222
0.5454545454545454
0.65
----------------------------------------
lambda = 0.1
0.4041666666666667
0.48148148148148145
0.5454545454545454
0.65
----------------------------------------
lambda = 0.2
0.42083333333333334
0.5
0.5681818181818182
0.65
----------------------------------------
lambda = 0.3
0.4791666666666667
0.5092592592592593
0.5681818181818182
0.75
----------------------------------------
lambda = 0.4
0.5125
0.5370370370370371
0.6363636363636364
0.8
----------------------------------------
lambda = 0.5
0.5583333333333333
0.5833333333333334
0.6363636363636364
0.9
----------------------------------------
lambda = 0.6
0.6041666666666666
0.6481481481481481
0.7272727272727273
0.9
----------------------------------------
lambda = 0.7
0.5958333333333333
0.6111111111111112
0.7045454545454546
0.8
----------------------------------------
lambda = 0.8
0.6
0.6018518518518519
0.6590909090909091
0.7
----------------------------------------


In [195]:
alpha = 0.5
threshold = 0.0
for i in tqdm(range(len(groups_n))):
    
    groups = groups_n[i]
    top_n_lists = top_n_lists_g[i]
    
    satisfactions = []
    for j in (range(len(groups))):
        
        group = groups[j]
        top_n_list = top_n_lists[j]
        
        high_ratings_t = np.zeros(top_n_size)
        high_ratings_u = np.zeros(len(group))
        
        for k in range(top_n_size):
            for l in range(len(group)):
                uid = group[l]
                tid = top_n_list[k]
                t = test[test['uid'] == uid]
                t = t[t['tid'] == tid]
                if len(t) > 0 and t.iloc[0]['rating'] > threshold:
                    high_ratings_t[k] += 1
                    high_ratings_u[l] += 1
        
#         satisfaction_t = 0
#         for k in range(top_n_size):
#             satisfaction_t += math.log(high_ratings_t[k]+1, 10) / math.log(len(group)+1, 10)
#         satisfaction_t /= top_n_size
        
#         satisfaction_u = 0
#         for k in range(len(group)):
#             satisfaction_u += math.log(high_ratings_u[k]+1, 10) / math.log(top_n_size+1, 10)
#         satisfaction_u /= len(group)
        satisfaction_t = math.log(max(high_ratings_t)+1, 10) / math.log(len(group)+1, 10)
        satisfaction_u = math.log(min(high_ratings_u)+1, 10) / math.log(top_n_size+1, 10)
        
        satisfaction = alpha*satisfaction_t + (1-alpha)*satisfaction_u
        satisfactions.append(satisfaction)
    
    avg_satisfaction = 0
    for satisfaction in satisfactions:
        avg_satisfaction += satisfaction
    avg_satisfaction /= len(satisfactions)
    print(avg_satisfaction)
        

0.4384689835799772
0.34093950544457624
0.3358646172870294
0.33769451038646087

